In [19]:
import numpy  as np
class Variable:
    def __init__(self,data):
        self.data = data

class Function:
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self,x):
        raise NotImplementedError()
    #抛出异常，告诉用户square等函数都是
    #集成于Function，需要自己实现

In [20]:
class Square(Function):
    def forward(self, x):
        return x**2
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [21]:
# 利用中心差分近似的数值分析方法来计算导数
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data-eps)
    x1 = Variable(x.data+eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data-y0.data)/(2*eps)

In [22]:
# 计算2.0处square 的导数
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [23]:
# 复合函数可以通过写函数实现，
# 实际上数值分析一般用于梯度检验反向传播是否错误
def f(x):
    A =Square()
    B =Exp()
    C = Square()
    return(C(B(A(x))))

x = Variable(np.array(0.5))
print(numerical_diff(f,x))

3.2974426293330694


In [24]:
# 反向传播的意义在于只要传播一次，就能求出
# 所有参数的导数
# 但是如果想要进行反向传播，就先要进行一次
# 正向传播 ，存储各函数的输入的变量值

![](../images/backward_trans.png)

In [25]:
# 扩展variable 记录Grad
class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None

class Function:
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input 
        #保存输入的变量
        return output
    def forward(self,x):
        raise NotImplementedError

    def backward(self,gy):
        raise NotImplementedError
        

In [3]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y

    def backward(self, gy):
        # gy 是输出方向传播来的导数
        x = self.input.data
        gx = 2*x*gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = gy*np.exp(x)
        return gx

In [27]:
A = Square()
B = Exp()
C = Square()
x = Variable(np.array(0.5))
a =A(x)
b =B(a)
y =C(b)
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [13]:
# 建立了计算图之间的连接
class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None
    def set_creator(self,func):
        self.creator = func
    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

class Function:
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    def forward(self,x):
        raise NotImplementedError

    def backward(self,gy):
        raise NotImplementedError
        

In [14]:
import numpy as np
A = Square()
B = Exp()
C = Square()
x = Variable(np.array(0.5))
a =A(x)
b =B(a)
y =C(b)

y.grad =np.array(1.0)
y.backward()
print(type(y))
print(x.grad)

<class '__main__.Variable'>
3.297442541400256


In [3]:
def square(x):
    return Square()(x)
def exp(x):
    return Exp()(x)
# 0维ndarray计算会变成ndarray以外的数据
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [7]:
# 之前是利用递归实现backward，现在改成用循环
class Variable:
    def __init__(self,data):
        if data is not None:
            if not isinstance(data,np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        # 要求输入一个ndarray的数组
        self.data = data
        self.grad = None
        self.creator = None
    def set_creator(self,func):
        self.creator = func
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs =[self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input ,f.output 
            x.grad =f.backward(y.grad)
            if self.creator is not None:
                funcs.append(x.creator)


class Function:
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    def forward(self,x):
        raise NotImplementedError

    def backward(self,gy):
        raise NotImplementedError
        

In [9]:
import unittest
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data,expected)

